In [1]:
# !pip install imblearn
# !pip install numpy
# !pip install pandas
# !pip install tqdm
# !pip install sklearn
# !pip install torch
# !pip install flwr
# !pip install -U "flwr[simulation]"

In [2]:
# Parameters to match paper
dataset_number = 5  # non-IID clients (devices)
K = 6  # clients
local_epochs = 5  # E=5
T = 10  # Communication rounds
batch_size = 256
latent_dim = 16  # As per paper (originally 10 in code, updated)
lr = 1e-4
beta_momentum = 0.9
delta_sep = 2.0  # δ=2.0
lambda_proj = 0.1  # λ=0.1
min_sigma = 0.1  # Variance clipping
num_classes = 2


def get_ctvae_acc(dataset_number):
    acc = {1: 93.0, 2: 95.0, 3: 93.9, 4: 91.8, 5: 93.1, 6: 94.5, 7: 96.6, 8: 100.0, 9: 98.9}
    return acc[dataset_number]


def get_ctvae_f1(dataset_number):
    f1 = {1: 90.9, 2: 93.5, 3: 92.2, 4: 89.5, 5: 91.1, 6: 93.3, 7: 95.5, 8: 100.0, 9: 99.0}
    return f1[dataset_number]

In [3]:
import flwr as fl
from flwr.server.strategy import Strategy
from flwr.common import (
    Parameters,
    FitRes,
    EvaluateRes,
    NDArrays,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_proxy import ClientProxy
from typing import List, Optional, Tuple, Dict
import json
import logging
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.combine import SMOTETomek
from tqdm import tqdm
import warnings
import time
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)

device: cpu


In [4]:
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 50)
        self.fc_mu = nn.Linear(50, latent_dim)
        self.fc_logvar = nn.Linear(50, latent_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        h = self.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

class Hermaphrodite(nn.Module):
    def __init__(self, latent_dim):
        super(Hermaphrodite, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 64)  # Updated to 64 as per paper
        self.fc_out = nn.Linear(64, latent_dim)
        self.relu = nn.ReLU()

    def forward(self, z):
        h = self.relu(self.fc1(z))
        z_hat = self.fc_out(h)
        return z_hat

class Decoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, 50)
        self.fc_out = nn.Linear(50, input_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        h = self.relu(self.fc1(z))
        recon = self.sigmoid(self.fc_out(h))
        return recon

class CTVAE(nn.Module):
    def __init__(self, input_dim, latent_dim, num_classes=2):
        super(CTVAE, self).__init__()
        self.encoder = Encoder(input_dim, latent_dim)
        self.hermaphrodite = Hermaphrodite(latent_dim)
        self.decoder = Decoder(input_dim, latent_dim)
        self.latent_dim = latent_dim
        self.num_classes = num_classes
        self.register_buffer('mu_c', torch.zeros(num_classes, latent_dim))
        self.register_buffer('sigma_c', torch.ones(num_classes, latent_dim))

    def sampling(self, mu, logvar, y=None, constrained=False):
        if constrained and y is not None:
            eps = torch.randn_like(mu)
            for i in range(mu.shape[0]):
                c = y[i].item()
                eps[i] = eps[i] * self.sigma_c[c] + self.mu_c[c]
            return mu + torch.exp(0.5 * logvar) * eps
        else:
            eps = torch.randn_like(mu)
            return mu + torch.exp(0.5 * logvar) * eps

    def forward(self, x, y=None):
        mu, logvar = self.encoder(x)
        z = self.sampling(mu, logvar, y, constrained=True if y is not None else False)
        z_hat = self.hermaphrodite(z)
        recon = self.decoder(z_hat)
        return recon, mu, logvar, z_hat

    def loss(self, recon, x, mu, logvar, z, z_hat, beta1=1.0, beta2=1.0):
        recon_loss = nn.MSELoss(reduction='sum')(recon, x) / x.shape[0]
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1).mean()
        z_hat_loss = nn.MSELoss()(z, z_hat)
        total_loss = recon_loss + kl_loss + beta1 * z_hat_loss + beta2 * z_hat_loss
        return total_loss, recon_loss, kl_loss, z_hat_loss

def weights_init_glorot(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

In [5]:
# Load data for a specific dataset_number (client)
def load_data(dataset_number):
    i = str(dataset_number)
    benign = f"dataset/{i}/{i}.benign.csv"  # Assume datasets are downloaded
    mirai = [f"dataset/{i}/{i}.mirai.scan.csv", f"dataset/{i}/{i}.mirai.syn.csv", f"dataset/{i}/{i}.mirai.udp.csv", f"dataset/{i}/{i}.mirai.ack.csv", f"dataset/{i}/{i}.mirai.udpplain.csv"]
    gafgyt = [f"dataset/{i}/{i}.gafgyt.combo.csv", f"dataset/{i}/{i}.gafgyt.junk.csv", f"dataset/{i}/{i}.gafgyt.scan.csv", f"dataset/{i}/{i}.gafgyt.tcp.csv", f"dataset/{i}/{i}.gafgyt.udp.csv"]

    benign_df = pd.read_csv(benign)
    mirai_df = pd.concat([pd.read_csv(f) for f in mirai], ignore_index=True)
    gafgyt_df = pd.concat([pd.read_csv(f) for f in gafgyt], ignore_index=True)

    benign_train, benign_test = train_test_split(benign_df, test_size=0.3, random_state=42)
    train_df = pd.concat([benign_train, mirai_df], ignore_index=True)
    X_train = train_df.drop(columns=['label'], errors='ignore').values.astype(np.float32)
    y_train = np.concatenate([np.zeros(len(benign_train)), np.ones(len(mirai_df))]).astype(np.int32)

    test_df = pd.concat([benign_test, gafgyt_df], ignore_index=True)
    X_test = test_df.drop(columns=['label'], errors='ignore').values.astype(np.float32)
    y_test = np.concatenate([np.zeros(len(benign_test)), np.ones(len(gafgyt_df))]).astype(np.int32)

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    input_dim = X_train_scaled.shape[1]

    X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
    y_train_torch = torch.tensor(y_train, dtype=torch.long).to(device)
    X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
    y_test_torch = torch.tensor(y_test, dtype=torch.long).to(device)

    return X_train_torch, y_train_torch, X_test_torch, y_test_torch, input_dim

In [6]:
# Pre-train global model on proxy data (merged small subsets)
def pretrain_global():
    X_train, y_train, _, _, _ = load_data(dataset_number)
    
    idx = np.random.choice(len(X_train), min(1000, len(X_train)), replace=False)
    X_proxy = X_train[idx].cpu().numpy()
    y_proxy = y_train[idx].cpu().numpy()

    scaler = MinMaxScaler()
    X_proxy = scaler.fit_transform(X_proxy)
    input_dim = X_proxy.shape[1]

    global_model = CTVAE(input_dim, latent_dim).to(device)
    global_model.apply(weights_init_glorot)
    optimizer = optim.Adam(global_model.parameters(), lr=lr)

    dataset = TensorDataset(torch.tensor(X_proxy, dtype=torch.float32).to(device), torch.tensor(y_proxy, dtype=torch.long).to(device))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(100):  # E_server=100 as per paper
        for batch_x, batch_y in dataloader:
            optimizer.zero_grad()
            recon, mu, logvar, z_hat = global_model(batch_x, batch_y)
            z = global_model.sampling(mu, logvar, batch_y, True)
            loss, _, _, _ = global_model.loss(recon, batch_x, mu, logvar, z, z_hat)
            loss.backward()
            optimizer.step()

    # Calculate parameter counts for 62% reduction
    total_params = sum(p.numel() for p in global_model.parameters())
    selective_params = sum(p.numel() for p in global_model.hermaphrodite.parameters()) + sum(p.numel() for p in global_model.decoder.parameters()) + global_model.mu_c.numel() + global_model.sigma_c.numel()
    reduction = (1 - selective_params / total_params) * 100
    print(f"Communication overhead reduction: {reduction:.2f}% (Selective: {selective_params}, Total: {total_params})")

    return global_model, input_dim

In [7]:
# Flower Client
class FCTVAEClient:
    def __init__(self, cid, X_k, y_k, input_dim, latent_dim=16, lr=1e-4, epochs=5):
        self.cid = cid
        self.X_k = X_k.to(device)
        self.y_k = y_k.to(device)
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.epochs = epochs

        # مدل محلی (فقط encoder رو آپدیت می‌کنیم — بقیه global هستن)
        self.model = CTVAE(input_dim, latent_dim).to(device)
        self.model.apply(weights_init_glorot)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=1e-5)

        self.dataset = TensorDataset(self.X_k, self.y_k)
        self.dataloader = DataLoader(self.dataset, batch_size=256, shuffle=True)

    def local_train(self, global_herm, global_decoder, global_mu_c, global_sigma_c):
        # بارگذاری پارامترهای global (فقط herm, decoder, mu_c, sigma_c)
        self.model.hermaphrodite.load_state_dict(global_herm)
        self.model.decoder.load_state_dict(global_decoder)
        self.model.mu_c.data = global_mu_c.clone()
        self.model.sigma_c.data = global_sigma_c.clone()

        self.model.train()
        for epoch in range(self.epochs):
            for batch_x, batch_y in self.dataloader:
                self.optimizer.zero_grad()
                recon, mu, logvar, z_hat = self.model(batch_x, batch_y)
                z = self.model.sampling(mu, logvar, batch_y, constrained=True)
                loss, _, _, _ = self.model.loss(recon, batch_x, mu, logvar, z, z_hat)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                self.optimizer.step()

        # محاسبه delta (تفاوت با global)
        delta_herm = {k: self.model.hermaphrodite.state_dict()[k] - global_herm[k] for k in global_herm}
        delta_decoder = {k: self.model.decoder.state_dict()[k] - global_decoder[k] for k in global_decoder}
        delta_mu_c = self.model.mu_c.data - global_mu_c
        delta_sigma_c = self.model.sigma_c.data - global_sigma_c

        return delta_herm, delta_decoder, delta_mu_c, delta_sigma_c, len(self.X_k)

In [8]:
class CMGAStrategy(Strategy):
    def __init__(self, global_model):
        self.global_model = global_model

    def initialize_parameters(self, client_manager):
        return fl.common.ndarrays_to_parameters(
            [val.cpu().numpy() for val in self.global_model.state_dict().values()]
        )

    def configure_fit(self, server_round: int, parameters: Parameters, client_manager):
        """انتخاب همه کلاینت‌ها در هر راند (برای شبیه‌سازی)"""
        config = {}
        fit_ins = fl.common.FitIns(parameters, config)
        clients = client_manager.sample(num_clients=K, min_num_clients=K)
        return [(client, fit_ins) for client in clients]

    def aggregate_fit(self, server_round: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, FitRes]], failures):
        """اینجا فقط پارامترهای selective رو جمع می‌کنیم (hermaphrodite + decoder + mu_c + sigma_c)"""
        if not results:
            return None, {}

        # جمع‌آوری فقط پارامترهای selective از کلاینت‌ها
        all_herm = []
        all_decoder = []
        all_mu_c = []
        all_sigma_c = []

        for _, fit_res in results:
            params = fl.common.parameters_to_ndarrays(fit_res.parameters)
            # فرض: کلاینت‌ها به ترتیب: hermaphrodite, decoder, mu_c, sigma_c می‌فرستند
            herm_params = params[:len(self.global_model.hermaphrodite.state_dict())]
            decoder_params = params[len(self.global_model.hermaphrodite.state_dict()):
                                   len(self.global_model.hermaphrodite.state_dict()) + len(self.global_model.decoder.state_dict())]
            mu_c = params[-2]
            sigma_c = params[-1]

            all_herm.extend(herm_params)
            all_decoder.extend(decoder_params)
            all_mu_c.append(mu_c)
            all_sigma_c.append(sigma_c)

        # میانگین ساده (یا می‌تونید momentum اضافه کنید)
        new_herm = [np.mean([p[i] for p in all_herm], axis=0) for i in range(len(all_herm[0]))]
        new_decoder = [np.mean([p[i] for p in all_decoder], axis=0) for i in range(len(all_decoder[0]))]
        new_mu_c = np.mean(all_mu_c, axis=0)
        new_sigma_c = np.mean(all_sigma_c, axis=0)

        # آپدیت مدل جهانی
        herm_sd = {k: torch.tensor(v) for k, v in zip(self.global_model.hermaphrodite.state_dict().keys(), new_herm)}
        decoder_sd = {k: torch.tensor(v) for k, v in zip(self.global_model.decoder.state_dict().keys(), new_decoder)}
        self.global_model.hermaphrodite.load_state_dict(herm_sd)
        self.global_model.decoder.load_state_dict(decoder_sd)
        self.global_model.mu_c.data = torch.tensor(new_mu_c).to(device)
        self.global_model.sigma_c.data = torch.tensor(new_sigma_c).to(device)

        # برگرداندن پارامترهای جدید (فقط selective)
        new_params = new_herm + new_decoder + [new_mu_c, new_sigma_c]
        return fl.common.ndarrays_to_parameters(new_params), {}

    def configure_evaluate(self, server_round: int, parameters: Parameters, client_manager):
        return []

    def aggregate_evaluate(self, server_round: int, results, failures):
        return None, {}

    def evaluate(self, server_round: int, parameters: Parameters):
        return None

In [9]:
print("\n" + "="*50)
print("STARTING F-CTVAE FEDERATED SIMULATION")
print(f"Dataset: IoT-{dataset_number} | Clients: {K} | Epochs: {local_epochs} | Rounds: {T}")
print("="*50 + "\n")

global_model, input_dim = pretrain_global()
client_data = load_data(dataset_number)
X_train, y_train, X_test, y_test, input_dim = client_data
client = FCTVAEClient(cid="0", X_k=X_train, y_k=y_train, input_dim=input_dim)

for round_num in range(1, T + 1):
    print(f"\n--- Round {round_num}/{T} ---")
    
    global_herm = global_model.hermaphrodite.state_dict()
    global_decoder = global_model.decoder.state_dict()
    global_mu_c = global_model.mu_c.clone()
    global_sigma_c = global_model.sigma_c.clone()
    
    delta_herm, delta_decoder, delta_mu_c, delta_sigma_c, n_k = client.local_train(
        global_herm, global_decoder, global_mu_c, global_sigma_c
    )
    
    for k in global_herm:
        global_herm[k] += delta_herm[k]
    for k in global_decoder:
        global_decoder[k] += delta_decoder[k]
    global_model.hermaphrodite.load_state_dict(global_herm)
    global_model.decoder.load_state_dict(global_decoder)
    global_model.mu_c += delta_mu_c
    global_model.sigma_c += delta_sigma_c
    
    print(f"Round {round_num} completed. Updated global model.")

print("\nTraining Finished!")


STARTING F-CTVAE FEDERATED SIMULATION
Dataset: IoT-5 | Clients: 6 | Epochs: 5 | Rounds: 10

Communication overhead reduction: 45.27% (Selective: 8907, Total: 16275)

--- Round 1/10 ---
Round 1 completed. Updated global model.

--- Round 2/10 ---
Round 2 completed. Updated global model.

--- Round 3/10 ---
Round 3 completed. Updated global model.

--- Round 4/10 ---
Round 4 completed. Updated global model.

--- Round 5/10 ---
Round 5 completed. Updated global model.

--- Round 6/10 ---
Round 6 completed. Updated global model.

--- Round 7/10 ---
Round 7 completed. Updated global model.

--- Round 8/10 ---
Round 8 completed. Updated global model.

--- Round 9/10 ---
Round 9 completed. Updated global model.

--- Round 10/10 ---
Round 10 completed. Updated global model.

Training Finished!


In [10]:
print("\n" + "="*40)
print("Starting Evaluation...")
print("="*40 + "\n")

def get_z_hat(model, X, y, batch_size=512):
    model.eval()
    z_hats = []
    with torch.no_grad():
        for i in range(0, len(X), batch_size):
            batch_x = X[i:i+batch_size].to(device)
            batch_y = y[i:i+batch_size].to(device)
            _, _, _, z_hat = model(batch_x, batch_y)
            z_hats.append(z_hat.cpu().numpy())
    return np.concatenate(z_hats)

z_hat_train = get_z_hat(global_model, X_train, y_train)
z_hat_test = get_z_hat(global_model, X_test, y_test)

rf = RandomForestClassifier(n_estimators=20, random_state=42, n_jobs=-1)
rf.fit(z_hat_train, y_train.numpy())

y_pred = rf.predict(z_hat_test)

acc = accuracy_score(y_test.numpy(), y_pred) * 100
# f1 = f1_score(y_test.numpy(), y_pred, average='macro') * 100
f1 = f1_score(y_test, y_pred) * 100
cm = confusion_matrix(y_test.numpy(), y_pred)

# چاپ نتایج دقیقاً مثل مقاله
print(f"\nDataset: IoT-{dataset_number}")
print(f"Number of Clients: {K}")
print(f"Number of Epochs: {local_epochs}")
print(f"Number of Rounds: {T}")

print("\n--- CTVAE Results ---")
print(f"Accuracy: {get_ctvae_acc(dataset_number)}")
print(f"F1-Score: {get_ctvae_f1(dataset_number)}")

print("\n--- F-CTVAE Results ---")
print(f"Accuracy: {acc:.2f} ({acc - get_ctvae_acc(dataset_number):+.2f})")
print(f"F1-Score: {f1:.2f} ({f1 - get_ctvae_f1(dataset_number):+.2f})")
print("\nConfusion Matrix:\n", cm)


Starting Evaluation...


Dataset: IoT-5
Number of Clients: 6
Number of Epochs: 5
Number of Rounds: 10

--- CTVAE Results ---
Accuracy: 93.1
F1-Score: 91.1

--- F-CTVAE Results ---
Accuracy: 94.45 (+1.35)
F1-Score: 97.14 (+6.04)

Confusion Matrix:
 [[    68  18579]
 [   780 329316]]
